In [ ]:
import sklearn.datasets as dataset
import numpy as np
iris = dataset.load_iris()

### train과 test를 나누는 메소드 ###
def train_test_split(dataset, name = 'mnist',ratio = 0.1):
    dataset_data = dataset['data']
    dataset_label = dataset['target']
    data_list = []
    label_list = []
    for data, label in zip(dataset_data, dataset_label):
      if label==0 or label==1:
        data_list.append(data)
        label_list.append(label)
    data_list = np.array(data_list)
    label_list = np.array(label_list)
    ### 순서대로 되어있으므로 train과 test에 label이 고르게 들어가기 위해서 중간값을 train, 나머지값을 test로 만든다
    train_data = data_list[int(data_list.shape[0]*ratio):int(data_list.shape[0]*(1-ratio)),]
    test_data = np.concatenate([data_list[:int(data_list.shape[0]*ratio),], data_list[int(data_list.shape[0]*(1-ratio)):,]])
    train_label = label_list[int(label_list.shape[0]*ratio):int(label_list.shape[0]*(1-ratio)),]
    test_label = np.concatenate([label_list[:int(label_list.shape[0]*ratio),], label_list[int(label_list.shape[0]*(1-ratio)):,]])
    
    return train_data, train_label, test_data, test_label

print('=======iris========')
train_data, train_label, test_data, test_label = train_test_split(iris)
print('train data : {} \t train label : {}'.format(train_data.shape, train_label.shape))
print('test data : {} \t test label : {}'.format(test_data.shape, test_label.shape))
print(train_label)
print(test_label)


In [ ]:
import numpy as np

class Network(object):
  def __init__(self, error_function='bce'):
    '''
    # random initialize 할때 사용 #
    # nan 뜨는 경우가 존재함 #
    self.weights = np.random.randn(4) # input shape이 4 이므로 이를 맞춰준다
    self.bias = np.random.randn(1) # bias 생성
    '''
    # zero initialize 할때 사용 #
    self.weights = np.zeros(4) # input shape이 4 이므로 이를 맞춰준다
    self.bias = np.zeros(1) # bias 생성

    self.lr = 0.001 # learning rate 생성
    self.epochs = 100 # epoch
    self.error_function = error_function # error function 지정
    
  def feedforward(self,x): # forward 단계이다
    x = self.weights.T.dot(x) + self.bias # w^t x + b = y -> sig(y) 하는 과정
    x = self.sigmoid(x)
    return x

  def backpropagation(self, x, pred, target):
    # back prropagation 단계로 보고서에 자세히 작성했으므로 생략하겠다
    if self.error_function == 'mse':
      error = self.mse_error(pred, target)
      for i in range(self.weights.shape[0]):
        dw = self.dev_mse_error(pred, target) * self.dev_sig(pred) * x[i]
        self.weights[i] -= self.lr * dw
      self.bias -= self.lr * self.dev_mse_error(pred, target) * self.dev_sig(pred)
    elif self.error_function == 'bce':
      error = self.binary_crossentropy(pred, target)
      for i in range(self.weights.shape[0]):
        dw = self.dev_binary_crossentropy(pred, target) * self.dev_sig(pred) * x[i]
        self.weights[i] -= self.lr * dw
      self.bias -= self.lr * self.dev_binary_crossentropy(pred, target) * self.dev_sig(pred)
    

  def predict(self,x):
    x = self.feedforward(x)
    if x>=0.5:
      return 1
    else:
      return 0

  def sigmoid(self, x):
    return 1.0 / (1.0 + np.exp(-x))

  def dev_sig(self, x):
    ### 보고서에 식 도출하는 과정 작성했음 ###
    return self.sigmoid(x) * (1- self.sigmoid(x))

  ### error계산할때는 float로 타입 변환 ###
  def mse_error(self, pred, target):
    target = target.astype('float')
    return (pred-target)**2
  
  def dev_mse_error(self, pred, target):
    target = target.astype('float')
    return 2*(pred-target)

  def binary_crossentropy(self, pred, target):
    target = target.astype('float')
    return -(target * np.log(pred) + (1-target) * np.log(1-pred))

  def dev_binary_crossentropy(self, pred, target):
    target = target.astype('float')
    #return ((target*(1-pred)) - ((1-target) * pred)) / ((pred*(1-pred)))
    return (pred-target) / (pred*(1-pred)) # 식 간추린거

  def accuracy(self, preds, targets):
    matrix = []
    for pred, target in zip(preds, targets):
      if pred==target:
        matrix.append(1)
      else:
        matrix.append(0)
    matrix = np.array(matrix)
    return matrix.sum()/len(matrix)

  def train(self,x, y):
    running_loss = 0
    for data, target in zip(x,y):
        pred = self.feedforward(data) # forward 진행
        if self.error_function=='mse':
            error = self.mse_error(pred, target)
        elif self.error_function=='bce':
            error = self.binary_crossentropy(pred, target)
        running_loss+=error[0]
        self.backpropagation(data, pred, target) # backward 진행 -> 업데이트 함
    loss = running_loss/len(x)
    return loss

  def test(self, x, y):
    pred = []
    for data in x:
        pred.append(slp.predict(data))
    score = self.accuracy(pred, y)
    return score

  def run(self, X_train, Y_train, X_test, Y_test):
    train_losses = []
    validation_score = []
    for epoch in range(self.epochs):
      loss = self.train(X_train, Y_train)
      train_losses.append(loss)
      if (epoch+1)%10==0: # 10번마다 validation 측정
          score = self.test(X_test, Y_test)
          validation_score.append(score)
          print('epoch : {} \t train loss : {:.5f} \t validation score : {:.5f}'.format(epoch+1, loss, score))

    epochs = np.arange(0,len(train_losses))
    plt.title('train loss')
    plt.plot(epochs, train_losses)
    plt.show()
    plt.title('validation score')
    epochs = np.arange(0,len(validation_score))
    plt.plot(epochs, validation_score)
    plt.show()

In [ ]:
# mse로 할때 #
import matplotlib.pyplot as plt

slp = Network(error_function='mse')
X_train, Y_train, X_test, Y_test = train_data, train_label, test_data, test_label
slp.run(X_train, Y_train, X_test, Y_test)

In [ ]:
# bce로 할때 #
import matplotlib.pyplot as plt

slp = Network(error_function='bce')
X_train, Y_train, X_test, Y_test = train_data, train_label, test_data, test_label
slp.run(X_train, Y_train, X_test, Y_test)